In [1]:
import pandas as pd
import sqlite3 as sql
import nltk

In [2]:
conn = sql.connect("track_metadata.db")
# conn2 = sql.connect("dat/mxm_dataset.db")
c = conn.cursor()
for row in c.execute("SELECT name FROM main.sqlite_master WHERE type='table';"):
    print(row)
c.execute("attach 'mxm_dataset.db' AS lyr;")
c.execute("attach 'lastfm_tags.db' AS tag;") 
for row in c.execute("SELECT name FROM lyr.sqlite_master WHERE type='table';"):
    print(row)
for row in c.execute("SELECT name FROM tag.sqlite_master WHERE type='table';"):
    print(row)
q = '''
SELECT * FROM songs
LIMIT 200;
'''
q2 = '''
SELECT * FROM lyrics
LIMIT 200;
'''
c1 = '''
ATTACH DATABASE "dat/mxm_dataset.db" AS conn3;
'''

c2 = '''
ATTACH DATABASE "dat/artist_term.db" AS conn3;
'''

songs = pd.read_sql("SELECT * FROM main.songs", conn)
lyrics = pd.read_sql("SELECT * FROM lyr.lyrics LIMIT 100", conn)
genres = pd.read_sql("SELECT * FROM tag.tags LIMIT 100", conn)
gtags = pd.read_sql("SELECT * FROM tag.tid_tag LIMIT 100", conn)

('songs',)
('words',)
('lyrics',)
('tags',)
('tids',)
('tid_tag',)


# Cleaning Begins!

In [3]:
#artist name, song title, release, all lowercase
songs["title"] = songs["title"].str.lower()
songs["release"] = songs["release"].str.lower()
songs["artist_name"] = songs["artist_name"].str.lower()

#drops artist hotttness metric; drops na vals
songs = songs.drop(columns = ["artist_hotttnesss", "track_7digitalid", "shs_perf", "shs_work", "artist_mbid"])

songs = songs.dropna()

#trim songs with years before 1975
songs = songs[songs.year > 1975]

#renaming the title column to "song_name" and rename "release" to "album_name"
songs = songs.rename(columns={"title":"song_name", "release":"album_name"})

In [4]:
#dropping based on familiarity being below 20
songs = songs[songs.artist_familiarity > .2]
songs = songs.sort_values(by = "artist_familiarity", ascending = False)

#removes rows with blank song entries
songs = songs[songs.song_name.str.len() > 1].sort_values(by="song_name")

#removes rows with years before 1975
songs = songs[songs.year > 1975]

#duplicate removal--removes all duplicate songs by matching artist id
songs = songs.sort_values(by = "artist_name")
songs["s/a"] = songs["song_name"]+songs["artist_id"] #song-artist concatenation column
songs = songs.drop_duplicates("s/a", keep = 'first')

 # TODO: if alternate versions of songs exist, keep the original only

In [5]:
#IMPLIMENT IRENE!

# Begin Merging Genre and Lyrics Section

In [6]:
#this section is to determine the most popular genre labels

"""sg = pd.read_sql("SELECT tag.tids.tid, tag.tags.tag FROM tag.tid_tag \
            INNER JOIN tag.tags ON tag.tags.ROWID==tag.tid_tag.tag \
            INNER JOIN tag.tids ON tag.tid_tag.tid==tag.tids.ROWID \
             LIMIT 1000000", conn)"""

sg = pd.read_sql("SELECT tag.tids.tid, tag.tags.tag FROM tag.tid_tag \
            INNER JOIN tag.tags ON tag.tags.ROWID==tag.tid_tag.tag \
            INNER JOIN tag.tids ON tag.tid_tag.tid==tag.tids.ROWID \
             ", conn)

data = pd.DataFrame()
sg['tag'] = sg.tag.str.lower()
gdict = sg['tag'].value_counts().to_dict()
populargenres = pd.DataFrame.from_dict(gdict, orient='index', columns = ['genre_count'])

In [9]:
songs.head(50)

,track_id,song_name,song_id,album_name,artist_id,artist_name,duration,artist_familiarity,year,s/a
354167,TRZRBYS128F92FF94F,break in case of anything,SOYDOCQ12A6D4F570B,myth takes,ARTW7I31187B9A4CA9,!!!,219.84608,0.746680,2007,break in case of anythingARTW7I31187B9A4CA9
957240,TRKXWOT128F92FF92A,heart of hearts,SOXTMKS12A6D4F5707,myth takes,ARTW7I31187B9A4CA9,!!!,362.91873,0.746680,2007,heart of heartsARTW7I31187B9A4CA9
614186,TRUKKNT128F92FF937,sweet life,SOGRWKZ12A6D4F5708,myth takes,ARTW7I31187B9A4CA9,!!!,226.69016,0.746680,2007,sweet lifeARTW7I31187B9A4CA9
840185,TRDDLOD128F92FF922,a new name,SOAWYQK12A6D4F5706,myth takes,ARTW7I31187B9A4CA9,!!!,294.94812,0.746680,2007,a new nameARTW7I31187B9A4CA9
866299,TROPNZV128F92FF93D,yadnus,SOKCVIN12A6D4F5709,myth takes,ARTW7I31187B9A4CA9,!!!,313.10322,0.746680,2007,yadnusARTW7I31187B9A4CA9
440898,TRAAVRJ128F92FF90A,myth takes,SOYHNTG12A6D4F5703,myth takes,ARTW7I31187B9A4CA9,!!!,143.98649,0.609539,2007,myth takesARTW7I31187B9A4CA9
84162,TRGCKLJ128F92FF90E,all my heroes are weirdos,SORUSXB12A6D4F5704,myth takes,ARTW7I31187B9A4CA9,!!!,184.08444,0.746680,2007,all my heroes are weirdosARTW7I31187B9A4CA9
228260,TRRKQQU128F92FF947,bend over beethoven,SOESDGT12A6D4F570A,myth takes,ARTW7I31187B9A4CA9,!!!,486.81751,0.746680,2007,bend over beethovenARTW7I31187B9A4CA9
780854,TRVFSRF128F92FF916,must be the moon,SORLTDE12A6D4F5705,myth takes,ARTW7I31187B9A4CA9,!!!,356.10077,0.746680,2007,must be the moonARTW7I31187B9A4CA9
951663,TRKSFBU128F92FF956,infinifold,SOLJPWC12A6D4F570C,myth takes,ARTW7I31187B9A4CA9,!!!,311.74485,0.746680,2007,infinifold ARTW7I31187B9A4CA9


In [ ]:
populargenres.columns=['genre','genre_count']

In [33]:
holdgenre = populargenres.index.get_values()
populargenres["genre"] = holdgenre

,genre_count,genre
rock,101071,rock
pop,69159,pop
alternative,55777,alternative
indie,48175,indie
electronic,46270,electronic


In [34]:
sg_frequency=sg.merge(populargenres,left_on='tag',right_on='genre',how='left')

In [35]:
sg_frequency.head()

,tid,tag,genre_count,genre
0,TRCCCYE12903CFF0E9,classic rock,25771,classic rock
1,TRCCCYE12903CFF0E9,progressive rock,13406,progressive rock
2,TRCCCFH12903CEBC70,blues,20474,blues
3,TRCCCFH12903CEBC70,memphis slim,19,memphis slim
4,TRCCCJT128F429FFF6,pop,69159,pop


In [36]:
df_songs_merge=songs.merge(sg_frequency,left_on='track_id',right_on='tid',how='left')

In [37]:
df_songs_merge=df_songs_merge[~df_songs_merge['tag'].isna()]

In [38]:
df_songs_merge

,track_id,song_name,song_id,album_name,artist_id,artist_name,duration,artist_familiarity,year,s/a,tid,tag,genre_count,genre
13,TRPMYJW128F9323194,camera,SOCKIWT12AB017F1A6,a house full of friends,ARI31A31187B9A7B6C,#poundsign#,391.00036,0.391269,2005,cameraARI31A31187B9A7B6C,TRPMYJW128F9323194,great covers,443.0,great covers
14,TRPMYJW128F9323194,camera,SOCKIWT12AB017F1A6,a house full of friends,ARI31A31187B9A7B6C,#poundsign#,391.00036,0.391269,2005,cameraARI31A31187B9A7B6C,TRPMYJW128F9323194,for my dancy mix,3.0,for my dancy mix
15,TRPMYJW128F9323194,camera,SOCKIWT12AB017F1A6,a house full of friends,ARI31A31187B9A7B6C,#poundsign#,391.00036,0.391269,2005,cameraARI31A31187B9A7B6C,TRPMYJW128F9323194,stereo cool,65.0,stereo cool
16,TRPMYJW128F9323194,camera,SOCKIWT12AB017F1A6,a house full of friends,ARI31A31187B9A7B6C,#poundsign#,391.00036,0.391269,2005,cameraARI31A31187B9A7B6C,TRPMYJW128F9323194,dream for me please,5.0,dream for me please
17,TRPMYJW128F9323194,camera,SOCKIWT12AB017F1A6,a house full of friends,ARI31A31187B9A7B6C,#poundsign#,391.00036,0.391269,2005,cameraARI31A31187B9A7B6C,TRPMYJW128F9323194,thissongmakesmewanttocrynotafraidtosharemyemot...,1.0,thissongmakesmewanttocrynotafraidtosharemyemot...
18,TRHAEMQ128F427C025,ayso,SOOWKJU12A58A7A6F2,sevensummers - international pop volume two,ARI31A31187B9A7B6C,#poundsign#,246.12526,0.391269,2000,aysoARI31A31187B9A7B6C,TRHAEMQ128F427C025,happy,12291.0,happy
19,TRHAEMQ128F427C025,ayso,SOOWKJU12A58A7A6F2,sevensummers - international pop volume two,ARI31A31187B9A7B6C,#poundsign#,246.12526,0.391269,2000,aysoARI31A31187B9A7B6C,TRHAEMQ128F427C025,-,53.0,-
20,TRHAEMQ128F427C025,ayso,SOOWKJU12A58A7A6F2,sevensummers - international pop volume two,ARI31A31187B9A7B6C,#poundsign#,246.12526,0.391269,2000,aysoARI31A31187B9A7B6C,TRHAEMQ128F427C025,the office girls,98.0,the office girls
23,TRJCBRC128F92D0FBC,todas las noches,SOKQBUR12A8C143F52,day by day,ARQ3NFE1187FB4801D,$tinkworx,253.93587,0.327675,2002,todas las nochesARQ3NFE1187FB4801D,TRJCBRC128F92D0FBC,future funk,39.0,future funk
24,TRWJHXP128F92D0FA2,creeping thru lower levels,SOHULMD12A8C140603,ain't-chit history,ARQ3NFE1187FB4801D,$tinkworx,237.71383,0.327675,2001,creeping thru lower levelsARQ3NFE1187FB4801D,TRWJHXP128F92D0FA2,808,83.0,808


# We just have to find a way to impute these vals now!

In [ ]:
alternative rock -->aleternative
hip-hop ---> hiphop
hip hop----> hiphop
hiphop
folk
pop
rock
indie
indie rock ---> indie
metal
heavy metal ---> metal
jazz
classic rock
elctronic 
electrocica ---> elctronic
house ---> electronic
techno ---> electronic
trance ---> electronic
electro --->electronic
soul
rap
country
blues
punk

